## Data Wrangling II:
### 1. Data Preparation
### 2. Encoding
### 3. Extra Reading

## 1. Data Preparation

### 1.1 Missing Data

In [1]:
import pandas as pd
import numpy as np

For data with float64 dtype, pandas uses the floating-point value `NaN` (Not a Number) to represent missing data.

In [2]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

In [3]:
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

The `isna` method gives us a Boolean Series with True where values are null:



In [4]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

The built-in Python None value is also treated as NA:



In [5]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

In [6]:
string_data

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [7]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [8]:
float_data = pd.Series([1, 2, None], dtype='float64')

In [9]:
float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [10]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

There are a few ways to filter out missing data. While you always have the option to do it by hand using pandas `.isna` and Boolean indexing, `dropna` can be helpful. On a Series, it returns the Series with only the nonnull data and index values:

In [11]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])

In [12]:
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [13]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [14]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, there are different ways to remove missing data. You may want to drop rows or columns that are all `NA`, or only those rows or columns containing any `NA`s at all. `dropna` by default drops any row containing a missing value:

In [15]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

In [16]:
data


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [17]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Passing `how="all"` will drop only rows that are all `NA`:



In [18]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [19]:
# Keep in mind that these functions return new objects by default and do not modify the contents 
# of the original object.
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis="columns"`:

In [20]:
data[4] = np.nan

In [21]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [22]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose you want to keep only rows containing at most a certain number of missing observations. You can indicate this with the thresh argument:

In [23]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))


In [24]:
df.iloc[:4, 1] = np.nan


In [25]:
df.iloc[:2, 2] = np.nan


In [26]:
df

,0,1,2
0,0.141455,NaN,NaN
1,2.167895,NaN,NaN
2,1.843211,NaN,-0.336979
3,-0.780970,NaN,-0.173498
4,1.160032,-0.341380,0.982564
5,-0.854402,-1.507094,-1.615927
6,1.348445,1.323033,3.305900


In [27]:
df.dropna()

,0,1,2
4,1.160032,-0.341380,0.982564
5,-0.854402,-1.507094,-1.615927
6,1.348445,1.323033,3.305900


In [28]:
df.dropna(thresh=2)

,0,1,2
2,1.843211,NaN,-0.336979
3,-0.780970,NaN,-0.173498
4,1.160032,-0.341380,0.982564
5,-0.854402,-1.507094,-1.615927
6,1.348445,1.323033,3.305900


Calling fillna with a constant replaces missing values with a certain value:

In [29]:
df.fillna(0)

,0,1,2
0,0.141455,0.000000,0.000000
1,2.167895,0.000000,0.000000
2,1.843211,0.000000,-0.336979
3,-0.780970,0.000000,-0.173498
4,1.160032,-0.341380,0.982564
5,-0.854402,-1.507094,-1.615927
6,1.348445,1.323033,3.305900


The same interpolation methods available for reindexing can be used with fillna:



In [30]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

In [31]:
df.iloc[2:, 1] = np.nan

In [32]:
df.iloc[4:, 2] = np.nan

In [33]:
df

,0,1,2
0,-0.003849,0.322100,-0.784366
1,1.295686,-0.038219,-1.474305
2,0.247656,NaN,-0.441182
3,-0.875631,NaN,0.601634
4,-1.057419,NaN,NaN
5,-1.368430,NaN,NaN


In [34]:
df.fillna(method="ffill")

,0,1,2
0,-0.003849,0.322100,-0.784366
1,1.295686,-0.038219,-1.474305
2,0.247656,-0.038219,-0.441182
3,-0.875631,-0.038219,0.601634
4,-1.057419,-0.038219,0.601634
5,-1.368430,-0.038219,0.601634


In [35]:
df.fillna(method="ffill", limit=2)

,0,1,2
0,-0.003849,0.322100,-0.784366
1,1.295686,-0.038219,-1.474305
2,0.247656,-0.038219,-0.441182
3,-0.875631,-0.038219,0.601634
4,-1.057419,NaN,0.601634
5,-1.368430,NaN,0.601634


With fillna you can do other things such as simple data imputation using the median or mean statistics:

In [36]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

In [37]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### 1.2 Data Transformation

#### Removing Duplicates

In [38]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                         "k2": [1, 1, 2, 3, 3, 4, 4]})

In [39]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [40]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicates` returns a DataFrame with rows where the duplicated array is `False` filtered out:



In [41]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates based only on the "k1" column:

In [42]:
data["v1"] = range(7)

In [43]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [44]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep="last"` will return the last one:

#### Transforming data with functions

Consider the following data collected about various kinds of meat:

In [45]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [46]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:

In [47]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

The `map` method on a Series accepts a function or dictionary-like object containing a mapping to do the transformation of values:

In [48]:
data["animal"] = data["food"].map(meat_to_animal)

In [49]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:

In [50]:
def get_animal(x):
    return meat_to_animal[x]

In [51]:
data["food"].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

#### Replacing Values

Filling in missing data with the `fillna` method is a special case of more general value replacement. `map` can be used to modify a subset of values in an object, but replace provides a simpler and more flexible way to do so. Let’s consider this Series:

In [52]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [53]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The `-999` values might be an indicator of values for missing data. To replace these with NA values that pandas understands, we can use replace, producing a new Series:



In [54]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the substitute value:


In [55]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:



In [56]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dictionary:



In [57]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## 2. Encoding

### 2.1 Categorical Encoding

The first case of categories is about data discretization. Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [58]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use `pandas.cut`:

In [59]:
bins = [18, 25, 35, 60, 100]

In [60]:
age_categories = pd.cut(ages, bins)

In [61]:
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see describes the bins computed by `pandas.cut`. Each bin is identified by a special (unique to pandas) interval value type containing the lower and upper limit of each bin:

In [62]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [63]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [64]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [65]:
pd.value_counts(age_categories)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

`pd.value_counts(categories)` are the bin counts for the result of `pandas.cut`.

### 2.2 One-Hot Encoding

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a dummy or indicator matrix, also known as one-hot encoding. 
If a column in a DataFrame has `k` distinct values, you would derive a matrix or DataFrame with `k` columns containing all 1s and 0s. pandas has a `pandas.get_dummies` function for doing this.

In [66]:
# For example de DF:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                       "data1": range(6)})

In [67]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [68]:
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


If a row in a DataFrame belongs to multiple categories, we have to use a different approach to create the dummy variables. For example, in the MovieLens 1M dataset:

In [69]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("movies.dat", sep="::",
                           header=None, names=mnames, engine="python")

In [70]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


pandas has a special Series method `str.get_dummies` that handles multiple group membership encoded as a delimited string, in this case, `|` for the genres:

In [71]:
dummies = movies["genres"].str.get_dummies("|")

In [72]:
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


### 2.3 Text representation

Suppose that you have a dataset with sentences like comments in a social media. A way to represent text as numeric data is using one-hot encoding using the words as categories. However, there are another approaches that could yield to better performance when training a Machine Learning model.

#### Bag Of Words
The first alternative approach to one-hot-encoding is the Bag of Words (BoW) model. Suppose you have the next toy dataset with 4 sentences:

`doc = ['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']`

If we map the words with IDs as: dog = 1, bites = 2, man = 3, meat = 4 , food = 5, eats = 6
Then the one-hot encoding scheme for the first sentence would be

`[[1 0 0 0 0 0], [0 1 0 0 0 0], [0 0 1 0 0 0]]`

The BoW model counts the frequencies of words in a sentence assigning the total of counts to the IDs. Thus, in the BoW model, the first sentence representantion can be stated as:

`[1 1 1 0 0 0]`.

This is because the first three words in the vocabulary appeared exactly once, and the last three did not appear at all.

#### scikit-learn API
In this and the next section we are going to use the sklearn package for the transformation of data, namely, text representation and scalings.

In [73]:
doc = ['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']

In [74]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [75]:
#Build a BOW representation for the corpus
bow_rep = count_vect.fit_transform(doc)

In [76]:
#Look at the vocabulary mapping
print("Our vocabulary: ", count_vect.vocabulary_)

Our vocabulary:  {'dog': 1, 'bites': 0, 'man': 4, 'eats': 2, 'meat': 5, 'food': 3}


In [77]:
#See the BOW rep for first 2 documents
print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
print("BoW representation for 'man bites dog: ",bow_rep[1].toarray())

BoW representation for 'dog bites man':  [[1 1 0 0 1 0]]
BoW representation for 'man bites dog:  [[1 1 0 0 1 0]]


In [78]:
#Get the representation using this vocabulary, for a new text
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':",
temp.toarray())

Bow representation for 'dog and dog are friends': [[0 2 0 0 0 0]]


Notice that 'dog' is the only word considered when counting because of its presence in the original voabulary.

Sometimes, we don’t care about the frequency of occurrence of words in text and we only want to represent whether a word exists in the text or not. In this case, use the `binary` argument set to `True`

In [79]:
count_vect = CountVectorizer(binary=True)
bow_rep_bin = count_vect.fit_transform(doc)
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':", temp.toarray())

Bow representation for 'dog and dog are friends': [[0 1 0 0 0 0]]


#### TF-IDF

`TF-IDF`, or term frequency–inverse document frequency, quantify the importance of a given word relative to other words in the document and in the corpus. 

`TF` (term frequency) measures how often a term or word occurs in a given document. Since different documents in the corpus may be of different lengths, a term may occur more often in a longer document as compared to a shorter document. To normalize these counts, we divide the number of occurrences by the length of the document. `IDF` (inverse document frequency) measures the importance of the term across a corpus. This solves the problem of common stop words like, is, are, am, etc. Now let's see a TF-IDF implementation:

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [81]:
tfidf = TfidfVectorizer()
bow_rep_tfidf = tfidf.fit_transform(doc)
print(tfidf.idf_) #IDF for all words in the vocabulary
print(tfidf.get_feature_names()) #All words in the vocabulary.

[1.51082562 1.22314355 1.51082562 1.91629073 1.22314355 1.91629073]
['bites', 'dog', 'eats', 'food', 'man', 'meat']


/home/saul/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [82]:
temp = tfidf.transform(["dog and man are friends"])
print("Tfidf representation for 'dog and man are friends':\n", temp.toarray())

Tfidf representation for 'dog and man are friends':
 [[0.         0.70710678 0.         0.         0.70710678 0.        ]]


### 2.4 Feature Scaling

Let's assume that we have two features where one feature is measured on a scale from 1 to 10 and the second feature is measured on a scale from 1 to 100,000, respectively. Some ML algorithms would fail to converge or would too long to converge. Now, there are two common approaches to bringing different features onto the same
scale: normalization and standardization. Normalization refers to the rescaling of the features to a range of `[0, 1]`, which is a special case of min-max scaling.

Using standardization, we center the feature columns at mean 0 with standard deviation 1 so that the feature  columns have the same parameters as a standard normal distribution (zero mean and unit variance).

We illustrate whis with the popular iris dataset:

In [83]:
from sklearn.datasets import load_iris

In [84]:
data = load_iris()

In [85]:
X = data['data']

Let's use the sklearn MinMaxScaler for normalization

In [86]:
from sklearn.preprocessing import MinMaxScaler

In [87]:
mms = MinMaxScaler()

In [88]:
X_norm = mms.fit_transform(X)

In [89]:
X[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [90]:
X_norm[:10]

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ]])

Similar to the MinMaxScaler, let's use the sklearn StandardScaler for standardization

In [91]:
from sklearn.preprocessing import StandardScaler

In [92]:
stdsc = StandardScaler()

In [93]:
X_std = stdsc.fit_transform(X)

In [94]:
X_std[:10]

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ],
       [-1.50652052,  0.09821729, -1.2833891 , -1.3154443 ],
       [-1.02184904,  1.24920112, -1.34022653, -1.3154443 ],
       [-0.53717756,  1.93979142, -1.16971425, -1.05217993],
       [-1.50652052,  0.78880759, -1.34022653, -1.18381211],
       [-1.02184904,  0.78880759, -1.2833891 , -1.3154443 ],
       [-1.74885626, -0.36217625, -1.34022653, -1.3154443 ],
       [-1.14301691,  0.09821729, -1.2833891 , -1.44707648]])

## 3. Extra Reading

### 3.1 sklearn tranformation pipelines
#### Suggested Reading: https://scikit-learn.org/stable/modules/compose.html#pipeline

### 3.2 Feature engineering
#### Suggested Reading: https://www.analyticsvidhya.com/blog/2021/03/step-by-step-process-of-feature-engineering-for-machine-learning-algorithms-in-data-science/